In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import datasets

In [2]:
iris = datasets.load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [3]:
"""Define name of the dataFrame as 'df' as this name would be used later in the decision tree . if you want to provide
   different name for the dataframe, please also chage the name of the same in the implementation part of decisio tree"""

"""all_features=np.array([i for i in df.columns]) , change the name '(from this df.columns to somethig like [name].columns )'
   of dataframe in the code , which is present in the implementation section of the decisionn tree """

"all_features=np.array([i for i in df.columns]) , change the name '(from this df.columns to somethig like [name].columns )'\n   of dataframe in the code , which is present in the implementation section of the decisionn tree "

In [4]:
df = pd.DataFrame(iris.data)

In [5]:
header= ['sl','sw','pl','pw']
df.columns=header

In [6]:
# function for converting numeric values to a distinct/ labels 
def label(val,*boundary):
    if val<boundary[0]:
        return 'a'
    elif val<boundary[1]:
        return 'b'
    elif val<boundary[2]:
        return 'c'
    else:
        return 'd'


In [7]:
# applying these labels to the dataset
def convert(df,oldFeatures):
    second = df[oldFeatures].mean()
    minimum = df[oldFeatures].min()
    first = (minimum+second)/2
    maximum = df[oldFeatures].max()
    third = (maximum + second)/2
    return df[oldFeatures].apply(label,args=(first,second,third))

In [8]:
# coverting the dataframe
df['SL']=convert(df,'sl')
df['SW']=convert(df,'sw')
df['PL']=convert(df,'pl')
df['PW']=convert(df,'pw')

In [9]:
df.drop(['sl','sw','pl','pw'],axis=1,inplace=True)
df.head()

,SL,SW,PL,PW
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a


In [41]:
'''for our decision tree we do the following:
we start with a node; 
case 1 = if the node is pure we dont need to split any further and output would be the only class present
case 2 = if there is no feature left to split upon , we provide the majority among the class as the output
If both the above condition are not met , then we find the best splitting condition and recursivelly call the split'''

'for our decision tree we do the following:\nwe start with a node; \ncase 1 = if the node is pure we dont need to split any further and output would be the only class present\ncase 2 = if there is no feature left to split upon , we provide the majority among the class as the output\nIf both the above condition are not met , then we find the best splitting condition and recursivelly call the split'

In [10]:
class TreeNode:
    def __init__(self, data,output):
#        Data = 1)it is the feature on which our node is going to split upon when fitting the training data.
#               2)its values is none for leaf node
#        Index = For assigning a preticular index to each node
#        Output = it represents the class with majority at that instance
#        Children = it is storing the children of node in the form of a dictionary for easy access
        self.data = data
        self.children = {}
        self.output = output
        self.index = -1
   
    def add_child(self,feature_value,obj):
#       adding values of children in a dictionary form 
        self.children[feature_value] = obj

In [35]:
class build_tree:
    def __init__(self):
        # this function provide the root node of our decision tree
        self.__root = None

    def frequency_counter(self,Y):
        # return the number of counts of unique classes in the dataset in the form of a dictionary
        count = {}
        for i in Y:
            if i in count:
                count[i]+=1
            else:
                count[i]=1
        return count


    def entropy(self,Y):
        # this function provides us with the entropy value of a given node in our tree
        freq = self.frequency_counter(Y)
        ent = 0
        total = len(Y)
        for i in freq:
            p = freq[i]/total
            ent += (-p)*math.log(p)
        return ent

    def gain_ratio(self,X,Y,selected_feature):
        # this function provides us with the gain ratio which is used as a metric for getting a better accuracy for out decision tree
        infoOriginal = self.entropy(Y) # "original" represents entropy before splitting
        infoF = 0 
        split_info = 0
        values = set(X[:,selected_feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y# Appending Y values as the last column in the newely created dataframe 
        starting_size = df.shape[0] 
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            infoF += (current_size/starting_size)*self.entropy(df1[df1.shape[1]-1])
            split_info += (-current_size/starting_size)*math.log(current_size/starting_size)

        # handling zero value error in case of splitInfo
        if split_info == 0 :
            return math.inf 
    

        info_gain = infoOriginal - infoF
        gain_ratio = info_gain / split_info
        return gain_ratio 



    def decision_tree(self,X,Y,features,level,classes, all_features=np.array([i for i in df.columns])):
#       classes = different class available for us based on dataset
#       level = represent depth of out tree
#       all_features = provides all the output labesl present in the dataframe
           

        
        
#       If the node consists of a single pure class
        if len(set(Y)) == 1:
            print("Level",level)
            output = None
            for i in classes:
                if i in Y:
                    output = i
                    print("Count of",i,"=",len(Y))
                else :
                    print("Count of",i,"=",0)
            print("Current Entropy is =  0.0")
            print("Reached leaf Node")
            print()
            return TreeNode(None,output)

#       If we run out of features to split upon
#       In this case the answer will be the class whose count is maximum for the given split
        if len(features) == 0:
            print("Level",level)
            freqs = self.frequency_counter(Y)
            output = None
            max_count = -math.inf
            for i in classes:
                if i not in freqs:
                    print("Count of",i,"=",0)
                else :
                    if freqs[i] > max_count :
                        output = i
                        max_count = freqs[i]
                    print("Count of",i,"=",freqs[i])

            print("Current Entropy  is =",self.entropy(Y))          

            print("Reached leaf Node")
            print()
            return TreeNode(None,output)

        
        
        # after the two base cases, we have to Find the best feature to split upon
        max_gain = -math.inf
        final_feature = None
        for f in features :
            current_gain = self.gain_ratio(X,Y,f)
            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f

        print("Level",level)
        freqs = self.frequency_counter(Y)
        output = None
        max_count = -math.inf
        for i in classes:
            if i not in freqs:
                print("Count of",i,"=",0)
            else :
                if freqs[i] > max_count :
                    output = i
                    max_count = freqs[i]
                print("Count of",i,"=",freqs[i])     
        print("Current Entropy is =",self.entropy(Y))
        print("Splitting on feature" , all_features[final_feature] , "with gain ratio" , max_gain)
        print()
        

            
        unique_values = set(X[:,final_feature]) # unique_values represents the unique values of the feature selected
        df = pd.DataFrame(X)        
        df[df.shape[1]] = Y# appending y values at the end of the data frame.
        current_node = TreeNode(final_feature,output)
        index  = features.index(final_feature)
        features.remove(final_feature)# removing the selected feature from the list.
        for i in unique_values:
            # Creating a new dataframe with value of selected feature = i
            df_new = df[df[final_feature] == i]
            # recursively calling on the splits
            node = self.decision_tree(df_new.iloc[:,0:df_new.shape[1]-1].values,df_new.iloc[:,df_new.shape[1]-1].values,features,level+1,classes)
            current_node.add_child(i,node)

        # Add the removed feature     
        features.insert(index,final_feature)

        return current_node
    
    def fit(self,X,Y):
        # Fits according to the given training data
        features = [i for i in range(len(X[0]))]
        classes = set(Y)
        level = 0
        self.root = self.decision_tree(X,Y,features,level,classes)
        
    def predictSingle(self,data,node):
        # predicts the class for a given testing point
        # when We reach, a leaf node
        if len(node.children) == 0 :
            return node.output
        val = data[node.data] # represents the value of feature on which the split is made       
        if val not in node.children :
            return node.output
        # Recursively call on the splits
        return self.predictSingle(data,node.children[val])

    def predict(self,X):
        # This function returns Y-predicted
        Y = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            Y[i] = self.predictSingle(X[i],self.root)
        return Y
    
    def score(self,X,Y):
        # this function returns the mean accuracy.
        Y_predicted = self.predict(X)
        counter = 0
        for i in range(len(Y_predicted)):
            if Y_predicted[i] == Y[i]:
                counter+=1
        return counter/len(Y_predicted)

In [36]:
from sklearn.model_selection import train_test_split
x= df.values
y = iris.target
X_train, X_test, y_train, y_test=train_test_split(x, y)
clf=build_tree()
clf.fit(X_train, y_train)
print()


Level 0
Count of 0 = 42
Count of 1 = 31
Count of 2 = 39
Current Entropy is = 1.0906896614201194
Splitting on feature PW with gain ratio 0.7227537963335781

Level 1
Count of 0 = 0
Count of 1 = 0
Count of 2 = 27
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 26
Count of 2 = 12
Current Entropy is = 0.6236548495681085
Splitting on feature PL with gain ratio 0.40276291385112956

Level 2
Count of 0 = 0
Count of 1 = 0
Count of 2 = 6
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 25
Count of 2 = 6
Current Entropy is = 0.4913274484505221
Splitting on feature SL with gain ratio 0.1397212077667352

Level 3
Count of 0 = 0
Count of 1 = 2
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 3
Count of 0 = 0
Count of 1 = 15
Count of 2 = 5
Current Entropy is = 0.5623351446188083
Splitting on feature SW with gain ratio 0.11009755776163734

Level 4
Count of 0 = 0
Count of 1 = 6
Count of 2 = 0
Current Entropy is =  0.0
Reach

In [37]:
print("Score=", clf.score(X_test, y_test))

Score= 0.9473684210526315


In [38]:
print('prediction on training data=' , clf.predict(X_train))

prediction on training data= [2 0 0 0 2 2 2 2 1 1 2 1 2 1 1 0 0 1 0 0 0 1 0 1 0 0 0 2 1 1 0 0 2 0 0 0 0
 2 2 1 1 0 2 2 1 1 2 2 1 0 1 1 1 1 1 2 0 0 2 1 0 0 0 2 0 0 2 1 2 1 0 2 1 1
 2 0 1 2 0 2 1 2 1 2 2 2 2 1 0 0 0 0 0 1 1 2 1 0 2 2 0 2 1 0 2 1 0 0 1 1 0
 0]


In [39]:
print('prediction on testing data=', clf.predict(X_test))

prediction on testing data= [1 0 2 2 2 1 1 0 2 1 1 2 0 1 2 1 2 0 1 1 1 1 1 0 1 1 1 1 2 1 0 2 0 1 1 1 1
 0]


## Implementation with Sklearn

In [16]:
from sklearn.tree import DecisionTreeClassifier

In [17]:
algo = DecisionTreeClassifier(criterion='entropy')

In [18]:
XTrain,XTest,yTrain,yTest = train_test_split(iris.data,iris.target,random_state=101)

In [19]:
algo.fit(XTrain,yTrain)

DecisionTreeClassifier(criterion='entropy')

In [20]:
predTrain = algo.predict(XTrain)
predTrain

array([1, 2, 0, 0, 0, 1, 1, 0, 2, 1, 0, 1, 1, 0, 2, 0, 1, 2, 1, 0, 0, 1,
       0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 1, 2, 1, 2, 0, 0, 0, 1, 0, 0, 0, 2,
       0, 0, 2, 2, 0, 0, 2, 2, 2, 0, 1, 2, 0, 0, 2, 2, 0, 1, 0, 2, 2, 0,
       2, 1, 1, 2, 0, 2, 0, 1, 2, 0, 1, 0, 2, 1, 2, 0, 2, 2, 2, 1, 0, 0,
       2, 1, 0, 1, 2, 2, 2, 1, 0, 2, 0, 2, 1, 2, 2, 0, 2, 1, 1, 1, 1, 1,
       0, 1])

In [21]:
predTest =algo.predict(XTest)
predTest

array([0, 0, 0, 1, 1, 2, 1, 1, 2, 0, 2, 0, 0, 2, 2, 1, 1, 1, 0, 2, 1, 0,
       1, 1, 1, 1, 1, 2, 0, 0, 2, 1, 2, 1, 2, 1, 1, 1])

In [22]:
from sklearn.metrics import confusion_matrix

In [23]:
print(confusion_matrix(yTrain,predTrain))

[[40  0  0]
 [ 0 33  0]
 [ 0  0 39]]


In [24]:
print(confusion_matrix(yTest,predTest))

[[10  0  0]
 [ 0 17  0]
 [ 0  1 10]]


In [25]:
from sklearn.metrics import accuracy_score

In [26]:
print(accuracy_score(yTest,predTest))

0.9736842105263158


In [27]:
from sklearn.tree import export_graphviz
import pydotplus

In [28]:
dot_data=export_graphviz(algo,out_file=None,feature_names=iris.feature_names,class_names =iris.target_names)
graph =pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf('iris.pdf')

True

In [29]:
# we are getting above 90 % accuracy with both the models, which is pretty good scores

In [40]:
# if in the preprocessing of the data , if we code our function like following:
# def convert(df,oldFeatures):
#     second = df[oldFeatures].mean()
#     first = 0.5 * second
#     third = 1.5 * second
#     return df[oldFeatures].apply(label,args=(first,second,third))

# in this case we would be getting accuracy around '0.8947368421052632' 